In [7]:
import numpy as np

fpath = 'test.txt'
fpath = 'input.txt'
# fpath = "test2.txt"

orthogonal_lines = []
lines = []
dims = [0, 0]
with open(fpath, 'r') as ifp:
    for l in [line.strip() for line in ifp.readlines()]:
        if l.startswith('#'):
            continue
        start, end = [ coords.strip().split(',') for coords in l.split('->') ]
        start = dict( x=int(start[0]), y=int(start[1]) )
        end = dict( x=int(end[0]), y=int(end[1]) )
        line = (start, end)
        # print(f"{line=}")
        
        # Get maximum dimensions to display grid
        xrange = [ start["x"], end["x"] ]
        yrange = [ start["y"], end["y"] ]
        
        # Adjust dimensions on grid
        if dims[0] < max(xrange):
            dims[0] = max(xrange)
        if dims[1] < max(yrange):
            dims[1] = max(yrange)
            
        # print(f"{dims=}")
        
        lines.append((start, end))
        # If orthogonal, store it specially!
        if start["x"] == end["x"] or start["y"] == end["y"]:
            orthogonal_lines.append((start, end))
            
dims = list(map(lambda x: x+1, dims))

In [8]:
grid = np.zeros(shape=dims, dtype=np.uint8)
for oline in orthogonal_lines:
    # print(f"{oline=}")
    start, end = oline
    # Adjust the starting points to deal with quirks in numpy indexing
    # NOTE(tparker): Slicing always starts with the lower value in numpy
    ys = sorted([start["y"], end["y"]])
    ys[1] += 1
    xs = sorted([start["x"], end["x"]])
    xs[1] += 1
    grid[ys[0]:ys[1], xs[0]:xs[1]] += 1

grid

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [9]:
# Count the number of points greater than 1
coords = np.where(grid > 1)
result = len(coords[0])
print(f"Number of points where 2 or more lines intersect: {result}")

Number of points where 2 or more lines intersect: 6572


In [10]:
# Part 2: Include diagonal lines!
lines = []
dims = [0, 0]
with open(fpath, 'r') as ifp:
    for l in [line.strip() for line in ifp.readlines()]:
        if l.startswith('#'):
            continue
        start, end = [ coords.strip().split(',') for coords in l.split('->') ]
        start = dict( x=int(start[0]), y=int(start[1]) )
        end = dict( x=int(end[0]), y=int(end[1]) )
        line = (start, end)
        # print(f"{line=}")
        
        # Get maximum dimensions to display grid
        xrange = [ start["x"], end["x"] ]
        yrange = [ start["y"], end["y"] ]
        
        # Adjust dimensions on grid
        if dims[0] < max(xrange):
            dims[0] = max(xrange)
        if dims[1] < max(yrange):
            dims[1] = max(yrange)
            
        lines.append(line)

dims = list(map(lambda x: x+1, dims))

In [11]:
# Construct grid
grid = np.zeros(shape=dims, dtype=np.uint8)

def isOrthogonal(line):
    start, end = line
    return start["x"] == end["x"] or start["y"] == end["y"]

def isDiagonal(line):
    start, end = line
    return abs(start["x"] - end["x"]) == abs(start["y"] - end["y"])

# Update points
for line in lines:
    # print(f"{line=}")
    start, end = line
    
    # Orthogonal lines
    if isOrthogonal(line):
        posx = start["x"]
        posy = start["y"]
        # If vertical (column)
        if start["x"] == end["x"]:
            while posy != end["y"]:
                grid[posy,posx] += 1
                # Move down
                if posy < end["y"]:
                    posy += 1
                # Move up
                if posy > end["y"]:
                    posy -= 1
            grid[posy,posx] += 1
            
        # If horizontal (row)
        if start["y"] == end["y"]:
            while posx != end["x"]:
                grid[posy,posx] += 1
                # Move right
                if posx < end["x"]:
                    posx += 1
                # Move left
                if posx > end["x"]:
                    posx -= 1
            grid[posy,posx] += 1
        
    # Diagonal lines
    elif isDiagonal(line):
        posx = start["x"]
        posy = start["y"]
        while posx != end["x"] and posy != end["y"]:
            grid[posy, posx] += 1
            # Move south-east
            if posx < end["x"] and posy < end["y"]:
                posx += 1
                posy += 1
            # Move south-west
            elif posx > end["x"] and posy < end["y"]:
                posx -= 1
                posy += 1
            # Move north-west
            elif posx > end["x"] and posy > end["y"]:
                posx -= 1
                posy -= 1
            # Move north-east
            elif posx < end["x"] and posy > end["y"]:
                posx += 1
                posy -= 1
        
        grid[posy, posx] += 1
        
            
print(f"{grid.shape}")
grid

(991, 991)


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [12]:
# Count the number of points greater than 1
coords = np.where(grid > 1)
result = len(coords[0])
print(f"Number of points where 2 or more lines intersect: {result}")

# Guess #1: 21517 (too high)
# Guess #2: 21486 (too high)
# Guess #3: 21466 (CORRECT)

Number of points where 2 or more lines intersect: 21466
